<a href="https://colab.research.google.com/github/trappertwo/rPPG-Toolbox/blob/main/%3CUpdated%3E_rPPG_on_compressed_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [ ]:
# Clone the repository from github if not already setup
import os

dir_path = '/content/rPPG-Toolbox'
if os.path.isdir(dir_path):
  print(f"The directory '{dir_path}' exists.")
else:
  !git clone https://github.com/trappertwo/rPPG-Toolbox.git
%cd '{dir_path}'
!ls '{dir_path}'

Cloning into 'rPPG-Toolbox'...
remote: Enumerating objects: 3118, done.
remote: Counting objects: 100% (1203/1203), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 3118 (delta 1129), reused 1063 (delta 1063), pack-reused 1915 (from 2)
Receiving objects: 100% (3118/3118), 333.09 MiB | 26.11 MiB/s, done.
Resolving deltas: 100% (2155/2155), done.
Updating files: 100% (289/289), done.
/content/rPPG-Toolbox
 config.py	       LICENSE		  setup.sh
 configs	       main.py		  tools
 dataset	       model_outputs	  unsupervised_methods
 evaluation	       neural_methods	 '<Updated>_rPPG_on_compressed_videos.ipynb'
 figures	       README.md	  wip
 final_model_release   requirements.txt


In [ ]:
# Explicitly install PyQt5 to use interactive plotting and avoid non-interactive backends
# See this relevant issue for more details: https://github.com/astral-sh/uv/issues/6893
# This requires Python 3.9+
!which python
!python --version
!pip install  --no-build-isolation PyQt5

/usr/local/bin/python
Python 3.11.11
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 16.1 MB/s eta 0:00:00


In [ ]:
### Setup using uv
### This gives an error when building the mamba package.
### Hence we need to directly install causal-conv1d and mamba-ssm.
### See README file under tools/mamba.
#!bash setup.sh uv
%%bash
rm -rf .venv
uv venv --python 3.8
source .venv/bin/activate
uv pip install --no-build-isolation setuptools wheel
uv pip install --no-build-isolation torch==2.1.2+cu121 torchvision==0.16.2+cu121 torchaudio==2.1.2+cu121 --index-url https://download.pytorch.org/whl/cu121
uv pip install --no-build-isolation -r requirements.txt
uv pip install --no-build-isolation causal-conv1d
uv pip install --no-build-isolation mamba-ssm


 Downloaded cpython-3.8.20-linux-x86_64-gnu
Using CPython 3.8.20
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate
Resolved 2 packages in 92ms
 Downloaded setuptools
Prepared 2 packages in 119ms
Installed 2 packages in 14ms
 + setuptools==75.3.0
 + wheel==0.45.1
Resolved 19 packages in 1.18s
 Downloaded networkx
 Downloaded torchaudio
 Downloaded pillow
 Downloaded torchvision
 Downloaded numpy
 Downloaded sympy
 Downloaded triton
 Downloaded torch
Prepared 19 packages in 39.34s
Installed 19 packages in 315ms
 + certifi==2022.12.7
 + charset-normalizer==2.1.1
 + filelock==3.13.1
 + fsspec==2024.6.1
 + idna==3.4
 + jinja2==3.1.4
 + markupsafe==2.1.5
 + mpmath==1.3.0
 + networkx==3.3
 + numpy==1.24.1
 + pillow==10.2.0
 + requests==2.28.1
 + sympy==1.13.1
 + torch==2.1.2+cu121
 + torchaudio==2.1.2+cu121
 + torchvision==0.16.2+cu121
 + triton==2.1.0
 + typing-extensions==4.12.2
 + urllib3==1.26.13
Resolved 114 packages in 1.28s
   Building jax==0.4.6
   Buildi

In [ ]:
### List packages in the virtual environment
!source .venv/bin/activate
!uv pip list

Package                      Version
---------------------------- ------------
absl-py                      2.1.0
asttokens                    3.0.0
astunparse                   1.6.3
backcall                     0.2.0
beautifulsoup4               4.13.3
cachetools                   5.5.1
causal-conv1d                1.0.0
certifi                      2022.12.7
charset-normalizer           2.1.1
comm                         0.2.2
cycler                       0.12.1
debugpy                      1.8.12
decorator                    5.1.1
einops                       0.8.1
executing                    2.2.0
filelock                     3.13.1
flatbuffers                  25.2.10
fsspec                       2024.10.0
gast                         0.4.0
gdown                        5.2.0
google-auth                  2.38.0
google-auth-oauthlib         1.0.0
google-pasta                 0.2.0
grpcio                       1.70.0
h5py                         2.10.0
huggingface-hub              

# PhysNet Training & Testing on Compressed Data


In [ ]:
### UBFC-rPPG dataset in drive folder
### (TODO) Figure out how to import data set directly into colab
from google.colab import drive

drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
### Train on (compressed) UBFC-RPPG dataset using PhysNet
###
### Update the YAML file under configs dir before running this
### TOOLBOX_MODE: "train_and_test"
### Train:
### TRAIN.DATA.DATA_PATH = "/content/drive/MyDrive/DATASETS/DATASET_100_1_train"
### TRAIN.DATA.CACHED_PATH: "/content/drive/MyDrive/DATASETS/DATASET_100_1_train_preprocess"
### TRAIN.DATA.DO_PREPROCESS: True  (only the first time running this)
###
### Valid:
### VALID.DATA.DATA_PATH = "/content/drive/MyDrive/DATASETS/DATASET_100_1_train"
### VALID.DATA.CACHED_PATH: "/content/drive/MyDrive/DATASETS/DATASET_100_1_train_preprocess"
### VALID.DATA.DO_PREPROCESS: True  (only the first time running this)
###
### Test:
### TEST.DATA.DATA_PATH = "/content/drive/MyDrive/DATASETS/DATASET_100_1_test"
### TEST.DATA.CACHED_PATH: "/content/drive/MyDrive/DATASETS/DATASET_100_1_test_preprocess"
### TEST.DATA.DO_PREPROCESS: True  (only the first time running this)
###
### *****To use compressed data, need to update the UBFC data loader to read from the .mp4 file
### See this: https://drive.google.com/drive/folders/1sQPD92SdJTdm0J2camd_9vAVG2iTdaA5
### Also, change DATASET_2 to DATASET_3

!source .venv/bin/activate && python main.py --config_file ./configs/train_configs/Compressed_UBFC-rPPG_UBFC-rPPG_UBFC-rPPG_PHYSNET_BASIC.yaml

=> Merging a config file from ./configs/train_configs/UBFC-rPPG_UBFC-rPPG_UBFC-PHYS_PHYSNET_BASIC.yaml
Configuration:
BASE: ['']
DEVICE: cuda:0
INFERENCE:
  BATCH_SIZE: 4
  EVALUATION_METHOD: FFT
  EVALUATION_WINDOW:
    USE_SMALLER_WINDOW: False
    WINDOW_SIZE: 10
  MODEL_PATH: 
LOG:
  PATH: runs/exp
MODEL:
  BIGSMALL:
    FRAME_DEPTH: 3
  DROP_RATE: 0.2
  EFFICIENTPHYS:
    FRAME_DEPTH: 10
  MODEL_DIR: runs/exp/UBFC-rPPG_SizeW72_SizeH72_ClipLength128_DataTypeDiffNormalized_DataAugNone_LabelTypeDiffNormalized_Crop_faceTrue_BackendHC_Large_boxTrue_Large_size1.5_Dyamic_DetFalse_det_len32_Median_face_boxFalse/PreTrainedModels
  NAME: Physnet
  PHYSFORMER:
    DIM: 96
    FF_DIM: 144
    NUM_HEADS: 4
    NUM_LAYERS: 12
    PATCH_SIZE: 4
    THETA: 0.7
  PHYSNET:
    FRAME_NUM: 128
  RESUME: 
  TSCAN:
    FRAME_DEPTH: 10
  iBVPNet:
    CHANNELS: 3
    FRAME_NUM: 160
NUM_OF_GPU_TRAIN: 1
TEST:
  DATA:
    BEGIN: 0.0
    CACHED_PATH: /content/drive/MyDrive/DATASETS/DATASET_100_1_test_preproc

# CONFIG COPY


In [ ]:
'''
BASE: ['']
TOOLBOX_MODE: "train_and_test" # "train_and_test"  or "only_test"
TRAIN:
  BATCH_SIZE: 4
  EPOCHS: 30
  LR: 9e-3
  MODEL_FILE_NAME: UBFC_UBFC_PURE_physnet_stan
  PLOT_LOSSES_AND_LR: True
  DATA:
    FS: 30
    DATASET: UBFC-rPPG
    DO_PREPROCESS: False            # if first time, should be true
    DATA_FORMAT: NCDHW
    DATA_PATH: "/content/drive/MyDrive/DATASETS/DATASET_100_1_train"                     # Raw dataset path, need to be updated
    CACHED_PATH: "/content/drive/MyDrive/DATASETS/DATASET_100_1_train_preprocess"    # Processed dataset save path, need to be updated
    EXP_DATA_NAME: ""
    BEGIN: 0.0
    END: 0.8
    PREPROCESS:
      DATA_TYPE: ['DiffNormalized'] #if use physnet, should be DiffNormalized
      DATA_AUG: ['None']    # 'None' or 'Motion' is supported, used if the data path points to an augmented dataset or requires augmentation
      LABEL_TYPE: DiffNormalized
      DO_CHUNK: True
      CHUNK_LENGTH: 128  #only support for factor of 512
      CROP_FACE:
        DO_CROP_FACE: True
        BACKEND: 'HC'    # HC for Haar Cascade, RF for RetinaFace
        USE_LARGE_FACE_BOX: True
        LARGE_BOX_COEF: 1.5
        DETECTION:
          DO_DYNAMIC_DETECTION: False
          DYNAMIC_DETECTION_FREQUENCY : 32
          USE_MEDIAN_FACE_BOX: False    # This should be used ONLY if dynamic detection is used
      RESIZE:
        H: 72
        W: 72
VALID:
  DATA:
    FS: 30
    DATASET: UBFC-rPPG
    DO_PREPROCESS: False                # if first time, should be true
    DATA_FORMAT: NCDHW
    DATA_PATH: "/content/drive/MyDrive/DATASETS/DATASET_100_1_train"                     # Raw dataset path, need to be updated
    CACHED_PATH: "/content/drive/MyDrive/DATASETS/DATASET_100_1_train_preprocess"    # Processed dataset save path, need to be updated
    EXP_DATA_NAME: ""
    BEGIN: 0.8
    END: 1.0
    PREPROCESS:
      DATA_TYPE: ['DiffNormalized'] #if use physnet, should be DiffNormalized
      DATA_AUG: ['None']    # 'None' or 'Motion' is supported, used if the data path points to an augmented dataset or requires augmentation
      LABEL_TYPE: DiffNormalized
      DO_CHUNK: True
      CHUNK_LENGTH: 128  #only support for factor of 512
      CROP_FACE:
        DO_CROP_FACE: True
        BACKEND: 'HC'    # HC for Haar Cascade, RF for RetinaFace
        USE_LARGE_FACE_BOX: True
        LARGE_BOX_COEF: 1.5
        DETECTION:
          DO_DYNAMIC_DETECTION: False
          DYNAMIC_DETECTION_FREQUENCY : 32
          USE_MEDIAN_FACE_BOX: False    # This should be used ONLY if dynamic detection is used
      RESIZE:
        H: 72
        W: 72
TEST:
  METRICS: ['MAE', 'RMSE', 'MAPE', 'Pearson', 'SNR', 'BA']
  USE_LAST_EPOCH: True                     # to use provided validation dataset to find the best epoch, should be false
  DATA:
    FS: 30
    DATASET: UBFC-rPPG
    DO_PREPROCESS: False                    # if first time, should be true
    DATA_FORMAT: NCDHW
    DATA_PATH: "/content/drive/MyDrive/DATASETS/DATASET_100_1_test"                     # Raw dataset path, need to be updated
    CACHED_PATH: "/content/drive/MyDrive/DATASETS/DATASET_100_1_test_preprocess"    # Processed dataset save path, need to be updated
    EXP_DATA_NAME: ""
    BEGIN: 0.0
    END: 1.0
    PREPROCESS :
      DATA_TYPE: ['DiffNormalized']         #if use physnet, should be DiffNormalized
      LABEL_TYPE: DiffNormalized
      DO_CHUNK: True
      CHUNK_LENGTH: 128                 #only support for factor of 512
      CROP_FACE:
        DO_CROP_FACE: True
        BACKEND: 'HC'    # HC for Haar Cascade, RF for RetinaFace
        USE_LARGE_FACE_BOX: True
        LARGE_BOX_COEF: 1.5
        DETECTION:
          DO_DYNAMIC_DETECTION: False
          DYNAMIC_DETECTION_FREQUENCY : 32
          USE_MEDIAN_FACE_BOX: False    # This should be used ONLY if dynamic detection is used
      RESIZE:
        H: 72
        W: 72
DEVICE: cuda:0
NUM_OF_GPU_TRAIN: 1
LOG:
  PATH: runs/exp
MODEL:
  DROP_RATE: 0.2
  NAME: Physnet
  PHYSNET:
    FRAME_NUM: 128
INFERENCE:
  BATCH_SIZE: 4
  EVALUATION_METHOD: "FFT"        # "FFT" or "peak detection"
  EVALUATION_WINDOW:
    USE_SMALLER_WINDOW: False        # Change this if you'd like an evaluation window smaller than the test video length
    WINDOW_SIZE: 10        # In seconds
  MODEL_PATH: ""
'''

'\nBASE: [\'\']\nTOOLBOX_MODE: "train_and_test" # "train_and_test"  or "only_test"\nTRAIN:\n  BATCH_SIZE: 4\n  EPOCHS: 30\n  LR: 9e-3\n  MODEL_FILE_NAME: UBFC_UBFC_PURE_physnet_stan\n  PLOT_LOSSES_AND_LR: True\n  DATA:\n    FS: 30\n    DATASET: UBFC-rPPG\n    DO_PREPROCESS: True            # if first time, should be true\n    DATA_FORMAT: NCDHW\n    DATA_PATH: "/content/drive/MyDrive/DATASETS/DATASET_100_1_train"                     # Raw dataset path, need to be updated\n    CACHED_PATH: "/content/drive/MyDrive/DATASETS/DATASET_100_1_train_preprocess"    # Processed dataset save path, need to be updated\n    FILE_LIST_PATH: "/data2/rppg_datasets/DataFileLists/"    # Path to store file lists, needs to be updated\n    EXP_DATA_NAME: ""\n    BEGIN: 0.0\n    END: 0.8\n    PREPROCESS:\n      DATA_TYPE: [\'DiffNormalized\'] #if use physnet, should be DiffNormalized\n      DATA_AUG: [\'None\']    # \'None\' or \'Motion\' is supported, used if the data path points to an augmented dataset or r

In [ ]:
'''
if 'None' in config_preprocess.DATA_AUG:
            # Utilize dataset-specific function to read video
            if os.path.isfile(os.path.join(data_dirs[i]['path'],"vid.avi")):
              frames = self.read_video(os.path.join(data_dirs[i]['path'],"vid.avi"))
            else:
              frames = self.read_video(
                os.path.join(data_dirs[i]['path'],"vid.mp4"))
        elif 'Motion' in config_preprocess.DATA_AUG:
            # Utilize general function to read video in .npy format
            frames = self.read_npy_video(
                glob.glob(os.path.join(data_dirs[i]['path'],'*.npy')))
        else:
            raise ValueError(f'Unsupported DATA_AUG specified for {self.dataset_name} dataset! Received {config_preprocess.DATA_AUG}.')
'''